In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
from sqlalchemy import create_engine, select, literal_column, text
from sqlalchemy.orm import sessionmaker
from sqlalchemy_schemadisplay import create_schema_graph
from sqlalchemy.sql import func, distinct

In [3]:
from sqla_schema import *

In [4]:
with open('SECRETS','r') as f:
    conn_info = json.load(f)

conn_string = f"postgresql://{conn_info['user']}:{conn_info['password']}@{conn_info['host']}:5432/{conn_info['dbname']}"

engine = create_engine(conn_string)
session = sessionmaker(engine)()

In [5]:
ephys_sessions = session.\
    query(Session.id,
          SessionType.name.label("session_type"),
          Mouse.sex,
          Mouse.date_of_birth,
          Genotype.name.label("full_genotype"),
          func.string_agg(distinct(Structure.abbreviation),literal_column("','")).label("all_structures")).\
    join(SessionType).\
    join(Mouse).\
    join(Genotype).\
    join(SessionProbe).\
    join(Channel).\
    join(Structure).\
    filter(SessionType.name == "brain_observatory_1.1").\
    filter(Mouse.sex == "M").\
    group_by(Session.id, SessionType.id, Mouse.id, Genotype.id)

print(ephys_sessions)

df = pd.read_sql(ephys_sessions.statement, ephys_sessions.session.bind)
df

SELECT session.id AS session_id, session_type.name AS session_type, mouse.sex AS mouse_sex, mouse.date_of_birth AS mouse_date_of_birth, genotype.name AS full_genotype, string_agg(DISTINCT structure.abbreviation, ',') AS all_structures 
FROM session JOIN session_type ON session_type.id = session.session_type_id JOIN mouse ON mouse.id = session.specimen_id JOIN genotype ON genotype.id = mouse.genotype_id JOIN session_probe ON session.id = session_probe.session_id JOIN channel ON session_probe.id = channel.session_probe_id JOIN structure ON structure.id = channel.structure_id 
WHERE session_type.name = %(name_1)s AND mouse.sex = %(sex_1)s GROUP BY session.id, session_type.id, mouse.id, genotype.id


,id,session_type,sex,date_of_birth,full_genotype,all_structures
0,715093703,brain_observatory_1.1,M,2018-09-23,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,"APN,CA1,CA3,DG,grey,LGd,LP,MB,PO,PoT,VISam,VIS..."
1,719161530,brain_observatory_1.1,M,2018-09-09,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,"APN,CA1,CA2,CA3,DG,Eth,grey,LGd,LP,MB,NOT,PO,P..."
2,721123822,brain_observatory_1.1,M,2018-09-06,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,"APN,CA1,CA3,DG,HPF,LGd,LGv,LP,MB,NOT,POL,PPT,P..."
3,732592105,brain_observatory_1.1,M,2018-10-01,wt/wt,"grey,VISal,VISl,VISp,VISpm,VISrl"
4,737581020,brain_observatory_1.1,M,2018-06-09,wt/wt,"grey,VISl,VISmma,VISp,VISpm,VISrl"
5,739448407,brain_observatory_1.1,M,2018-09-19,wt/wt,"grey,VIS,VISam,VISl,VISp,VISrl"
6,742951821,brain_observatory_1.1,M,2018-06-28,wt/wt,"grey,VIS,VISal,VISl,VISp,VISpm,VISrl"
7,743475441,brain_observatory_1.1,M,2018-06-27,wt/wt,"APN,CA1,CA3,DG,Eth,HPF,LGd,LP,MGv,PIL,PO,PP,Pr..."
8,744228101,brain_observatory_1.1,M,2018-05-26,wt/wt,"APN,CA1,CA2,CA3,DG,Eth,IGL,LGd,LP,POL,SGN,SUB,..."
9,750332458,brain_observatory_1.1,M,2018-06-26,wt/wt,"CA1,CA3,DG,grey,IGL,IntG,LGd,VISal,VISam,VISl,..."


In [6]:
for r in session.query(UnitSpikeTimes).limit(3):
    st = np.array(r.spike_times)
    print(r.unit_id, st)

950922146 [  31.49003496   32.92370245   35.5053706  ... 9578.0161067  9578.03900672
 9578.12100675]
950922041 [  26.97639903   27.05796574   27.11143244 ... 9577.51027313 9577.81670661
 9577.89237331]
950922383 [  26.97859903   27.00989904   27.05436574 ... 9578.11914009 9578.12497342
 9578.13414009]
